In [1]:
from __future__ import absolute_import
from __future__ import print_function

import os
import datetime
from shutil import copyfile

In [2]:
from utils import import_test_configuration, set_sumo, set_test_path, set_train_path
from fixed_time import *

from generator import TrafficGenerator
from visualization import Visualization

In [3]:



if __name__ == "__main__":
    config = import_test_configuration(config_file='testing_settings.ini')
    sumo_cmd = set_sumo(config['gui'], config['sumocfg_file_name'], config['max_steps'])
#     model_path, plot_path = set_test_path(config['models_path_name'], config['model_to_test'])
    plot_path = set_train_path(config['models_path_name'])
    
    
    #PARAMETERS
    fixed_green_time = 30
    
    
    
    TrafficGen = TrafficGenerator(
        config['max_steps'], 
        config['n_cars_generated'],
        config['penetration_rate']
    )
    
    
    Visualization = Visualization(
        plot_path, 
        dpi=96
    )
    
    
    Simulation = FixedTimeTestSimulation(
        TrafficGen,
        sumo_cmd,
        config['max_steps'],
        fixed_green_time,
        config['yellow_duration'],
        config['num_actions']
    )
    
    
    print('\n----- Test episode')
    simulation_time = Simulation.run(config['episode_seed'])  # run the simulation
    print('Simulation time:', simulation_time, 's')

    print("----- Testing info saved at:", plot_path)

    copyfile(src='testing_settings.ini', dst=os.path.join(plot_path, 'testing_settings.ini'))

#     Visualization.save_data_and_plot(data=Simulation.reward_episode, filename='reward', xlabel='Action step', ylabel='Reward')
    Visualization.save_data_and_plot(data=Simulation.queue_length_episode, filename='queue', xlabel='Step', ylabel='Queue lenght (vehicles)')

    


----- Test episode
Simulating...
Simulation time: 54.5 s
----- Testing info saved at: C:\Users\Chantal\Documents\Deep-QLearning-Agent-for-Traffic-Signal-Control\TLCS\models\model_171\
